In [3]:
import pyspark
from pyspark.sql import SparkSession
import seaborn as sns
import pandas as pd

spark = SparkSession.builder \
    .appName("MyApp") \
    .getOrCreate()

In [4]:
df = spark.read.csv("./originalDataSets/ecommerce_customer_data_large.csv", header=True, inferSchema=True)
df.show()

+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+--------------+---+------+-----+
|Customer ID|      Purchase Date|Product Category|Product Price|Quantity|Total Purchase Amount|Payment Method|Customer Age|Returns| Customer Name|Age|Gender|Churn|
+-----------+-------------------+----------------+-------------+--------+---------------------+--------------+------------+-------+--------------+---+------+-----+
|      44605|2023-05-03 21:30:02|            Home|          177|       1|                 2427|        PayPal|          31|    1.0|   John Rivera| 31|Female|    0|
|      44605|2021-05-16 13:57:44|     Electronics|          174|       3|                 2448|        PayPal|          31|    1.0|   John Rivera| 31|Female|    0|
|      44605|2020-07-13 06:16:57|           Books|          413|       1|                 2345|   Credit Card|          31|    1.0|   John Rivera| 31|Female|    0|
|      44605|202

In [15]:
from pyspark.sql.functions import col, sum, min, max, count, isnan, when, collect_set, countDistinct
from functools import reduce

numeric_cols = [c for c, t in df.dtypes if t in ['int', 'double', 'float']]
non_numeric_cols = [c for c in df.columns if c not in numeric_cols]
total_nulls = reduce(lambda a, b: a + b, [count(when(col(c).isNull() | (isnan(c) if c in numeric_cols else col(c).isNull()), c)) for c in df.columns]).alias("Total Null Values")

statistics = df.select(
    count("*").alias("Total Orders"),
    sum(col("Total Purchase Amount")).alias("Total Revenue"),
    min(col("Purchase Date")).alias("Earliest Purchase Date"),
    max(col("Purchase Date")).alias("Latest Purchase Date"),
    total_nulls,
    countDistinct(col("Product Category")).alias("Total Categories"),
    collect_set(col("Payment Method")).alias("Payment Methods"),
    count(when(col("Gender") == "Male", True)).alias("Orders by Male"),
    count(when(col("Gender") == "Female", True)).alias("Orders by Female"),
    sum(col("Quantity")).alias("Total Quantities Ordered")
)

statistics.show()

+------------+-------------+----------------------+--------------------+-----------------+----------------+--------------------+--------------+----------------+------------------------+
|Total Orders|Total Revenue|Earliest Purchase Date|Latest Purchase Date|Total Null Values|Total Categories|     Payment Methods|Orders by Male|Orders by Female|Total Quantities Ordered|
+------------+-------------+----------------------+--------------------+-----------------+----------------+--------------------+--------------+----------------+------------------------+
|      250000|    681346299|   2020-01-01 00:07:26| 2023-09-13 18:42:49|            47382|               4|[Credit Card, Pay...|        125676|          124324|                  751234|
+------------+-------------+----------------------+--------------------+-----------------+----------------+--------------------+--------------+----------------+------------------------+

